In [1]:
#setup imports and make ure the files we care about exist
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os.path

features_path = "/home/jaywalker/MachineLearning/UNSW-NB15-CSV/NUSW-NB15_features.csv"
print("Features file found!" if os.path.isfile("/home/jaywalker/MachineLearning/UNSW-NB15-CSV/NUSW-NB15_features.csv") else "")

Features file found!


In [2]:
#quick peek at the features and cleaning up column names for easier indexing
features_df = pd.read_csv(features_path, encoding="latin-1")
for i in range(len(features_df.columns.values)):
    features_df.columns.values[i] = str(features_df.columns.values[i]).strip().lower()
    
    
print(features_df.columns) #cleaned up column names

#lower case all the types
for i in range(len(features_df)):
    features_df.loc[i, ['type']] = str(features_df['type'][i]).strip().lower()
    features_df.loc[i, ['name']] = str(features_df['name'][i]).strip().lower()

print(features_df[['name', 'type']])

Index(['no.', 'name', 'type', 'description'], dtype='object')
                name       type
0              srcip    nominal
1              sport    integer
2              dstip    nominal
3             dsport    integer
4              proto    nominal
5              state    nominal
6                dur      float
7             sbytes    integer
8             dbytes    integer
9               sttl    integer
10              dttl    integer
11             sloss    integer
12             dloss    integer
13           service    nominal
14             sload      float
15             dload      float
16             spkts    integer
17             dpkts    integer
18              swin    integer
19              dwin    integer
20             stcpb    integer
21             dtcpb    integer
22           smeansz    integer
23           dmeansz    integer
24       trans_depth    integer
25       res_bdy_len    integer
26              sjit      float
27              djit      float
28        

In [3]:
#quick peek at the data
training_set_path = "/home/jaywalker/MachineLearning/UNSW-NB15-CSV/train_test/UNSW_NB15_training-set.csv"
training_df = pd.read_csv(training_set_path, encoding="latin-1")
print(training_df[:1])
#Of COURSE this file is organized differently than the features file describes.
#Why would I expect differently?

   id       dur proto service state  spkts  dpkts  sbytes  dbytes        rate  \
0   1  0.000011   udp       -   INT      2      0     496       0  90909.0902   

   ...    ct_dst_sport_ltm  ct_dst_src_ltm  is_ftp_login  ct_ftp_cmd  \
0  ...                   1               2             0           0   

   ct_flw_http_mthd  ct_src_ltm  ct_srv_dst  is_sm_ips_ports  attack_cat  \
0                 0           1           2                0      Normal   

   label  
0      0  

[1 rows x 45 columns]


In [4]:
#so we'll use a file from the "full" dataset instead
nb15_1_path = "/home/jaywalker/MachineLearning/PacketGAN/UNSW-NB15_1_clean.csv"
packet_data_df = pd.read_csv(nb15_1_path, encoding="latin-1", names=features_df['name'], header=None)
print(packet_data_df.columns)

/home/jaywalker/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'sload', 'dload',
       'spkts', 'dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'sjit', 'djit', 'stime',
       'ltime', 'sintpkt', 'dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'label'],
      dtype='object')


In [5]:
#for each feature of type "nominal" or "integer" count how many classes exist
#print(packet_data_df['label'].unique()) #identify the different values

for label, feature_type in features_df[['name', 'type']].values:
    nunique = packet_data_df[packet_data_df['label'] == 1][label].nunique()
    if nunique < 20:
        value_list = packet_data_df[packet_data_df['label'] == 1][label].unique().tolist()
        print(label + ": " , end='')
        print(value_list, end='')
        print(" type: " + str(feature_type))
    else:
        print(label + ": " + str(nunique) + " type: " + str(feature_type))
        
        
#how many attack packets do we have compared to non-attack packets?
print("Normal packets: ", len(packet_data_df[packet_data_df['label'] == 0].index))
print("Attack packets: ", len(packet_data_df[packet_data_df['label'] == 1].index))

srcip: ['175.45.176.0', '175.45.176.1', '175.45.176.2', '175.45.176.3'] type: nominal
sport: 9607 type: integer
dstip: ['149.171.126.15', '149.171.126.14', '149.171.126.10', '149.171.126.13', '149.171.126.18', '149.171.126.19', '149.171.126.11', '149.171.126.17', '149.171.126.12', '149.171.126.16'] type: nominal
dsport: 786 type: integer
proto: 129 type: nominal
state: ['INT', 'CON', 'REQ', 'FIN', 'ACC', 'CLO'] type: nominal
dur: 8748 type: float
sbytes: 2604 type: integer
dbytes: 2410 type: integer
sttl: [254, 0, 62, 255, 63] type: integer
dttl: [0, 60, 252, 253] type: integer
sloss: 186 type: integer
dloss: 200 type: integer
service: ['-', 'dhcp', 'dns', 'http', 'ftp-data', 'smtp', 'pop3', 'ftp', 'snmp', 'radius', 'ssl', 'ssh', 'irc'] type: nominal
sload: 9501 type: float
dload: 8654 type: float
spkts: 225 type: integer
dpkts: 245 type: integer
swin: [0, 255] type: integer
dwin: [0, 255] type: integer
stcpb: 8584 type: integer
dtcpb: 8552 type: integer
smeansz: 1072 type: integer
dme

In [6]:
#what is the maximum port value?
#attack_sports = packet_data_df[packet_data_df['label'] == 1]['sport']
for k in range(0, packet_data_df['sport'].shape[0]):
    if (isinstance(packet_data_df.loc[k, 'sport'], str)):
        packet_data_df.loc[k, 'sport'] = int(packet_data_df.loc[k, 'sport'])
        
#did we convert all the strings?
for k in range(0, packet_data_df['sport'].shape[0]):
    if (isinstance(packet_data_df.loc[k, 'sport'], str)):
        print(packet_data_df.loc[k, 'sport'])
        
#print(packet_data_df.loc[k, 'sport'])
#sorted_ports = attack_sports.sort_values(ascending=False)

In [7]:
#How can we encode these various features, many of which are discrete integers?
#One-hot or Binary encoding seems logical, using Binary coding to keep things compact.

#Returns a list where each element are a 1 or 0, determining the binary encoding of value with
#at least bits number of bits. If the value cannot be encoding with the requested number of bits,
#None will be returned.
def binary_encode(value, bits):
    encoding = []
    while value != 0:
        encoding.append(value % 2)
        value //= 2
        
    if bits < len(encoding):
        return None #couldn't represent with requested number of bits
    
    while len(encoding) < bits:
        encoding.append(0)
    
    encoding.reverse()
    return encoding
        
print(binary_encode(7, 4)) #returns [0,1,1,1]
print(binary_encode(255, 2)) #returns None

[0, 1, 1, 1]
None


In [8]:
#try converting an ip into a bit array
ip_as_bits = []
for byte in packet_data_df['srcip'][0].split('.'):
    ip_as_bits += binary_encode(int(byte), 8)
    
print(ip_as_bits)

[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [9]:
#lets see what some of the other relevant fields look like
timestamp = packet_data_df['stime'][0]
print(timestamp)
nbits = 36
print(binary_encode(timestamp, nbits))
#can all the timestamps be represented with fewer bits?
for k in packet_data_df['stime']:
    if binary_encode(k, nbits) is None:
        print("Couldn't map all the timestamps!")
        break

1421927415
[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]


In [10]:
#what features do I care about?
#all the non-aggregate features that are some combination of the other features
#either directly in an example or temporal combinations, since these should ostensibly
#be discovered by the GAN
features_to_use = features_df[:39]
print(features_to_use[['name', 'type']])

                name       type
0              srcip    nominal
1              sport    integer
2              dstip    nominal
3             dsport    integer
4              proto    nominal
5              state    nominal
6                dur      float
7             sbytes    integer
8             dbytes    integer
9               sttl    integer
10              dttl    integer
11             sloss    integer
12             dloss    integer
13           service    nominal
14             sload      float
15             dload      float
16             spkts    integer
17             dpkts    integer
18              swin    integer
19              dwin    integer
20             stcpb    integer
21             dtcpb    integer
22           smeansz    integer
23           dmeansz    integer
24       trans_depth    integer
25       res_bdy_len    integer
26              sjit      float
27              djit      float
28             stime  timestamp
29             ltime  timestamp
30      

Feature encoding
-------------------------

I will encode the integer based features using a binary representation, using the minimum number of bits to represent the max value plus one bit. Float based parameters will be scaled in a typical manner.

IP addresses in particular are a special case, since each field is represending a collection of 4 bytes. These addresses will be represented as 32 bits, since this is the native representation and seems appropriate for this task.

In [71]:
def get_minimum_bits(value):
    min_bits = 1
    while binary_encode(value, min_bits) is None:
        min_bits += 1
        
    return min_bits

def build_input_feature_tensor(packet_data_dict):
    input_features = []
    
    srcip_segments = str(packet_data_dict['srcip']).split('.')
    srcip_bits = []
    for segment in srcip_segments:
        for k in binary_encode(int(segment), 8):
            srcip_bits.append(k)
    
    dstip_segments = str(packet_data_dict['dstip']).split('.')
    dstip_bits = []
    for segment in dstip_segments:
        for k in binary_encode(int(segment), 8):
            dstip_bits.append(k)
            
    sport = binary_encode(int(packet_data_dict['sport']), 16)#get_minimum_bits(int(packet_data_dict['sport'])) + 1)
    dport = binary_encode(int(packet_data_dict['dsport']), 16)#get_minimum_bits(int(packet_data_dict['dsport'])) + 1)
    
    #TODO need to encode the rest of the features buuuuuttttt that can come later.
    
    input_features += srcip_bits + dstip_bits + sport + dport
    
    return torch.tensor(input_features, dtype=torch.float32)
        
X = build_input_feature_tensor(packet_data_df.loc[0,:].to_dict())

#just playing with tensors here to figure out what I'm doing!
print(X)
print(X.shape)
X_seq = torch.tensor(()).new_zeros([5,1,X.shape[0]])
print(X_seq.shape)

#wow, I did not expect this to work!
X_seq[:,0,:] = X
#print(X_seq)

def build_input_sequence_tensor(packet_data_df, sequence_length):
    example_feature_vector = build_input_feature_tensor(packet_data_df.loc[0,:].to_dict())
    seq_out = torch.tensor(()).new_zeros([sequence_length, 1, example_feature_vector.shape[0]])
    
    for i in range(0, sequence_length):
        #print(seq_out.shape)
        seq_out[i,0,:] = build_input_feature_tensor(packet_data_df.loc[i,:].to_dict())
        
    return seq_out

X_seq = build_input_sequence_tensor(packet_data_df, 5)
print(X_seq)

tensor([0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
torch.Size([96])
torch.Size([5, 1, 96])
tensor([[[0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
          0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
          0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
  

In [82]:
#I shouldn't actually do the training in this notebook, this is mostly a test to see if I've prepared
#the features correctly for input to some RNN network.

#MODELS: Define Generator model and Discriminator model
#For the time being, this will be a one-layer RNN that is the same width as the input feature tensor

class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Generator, self).__init__()
        self.hidden_size = hidden_size
        self.h0 = self.init_hidden_state()
        self.gru = nn.GRU(input_size, hidden_size)
        self.map = nn.Linear(input_size, input_size)
        self.f = f

    def forward(self, x):
        x, self.h0 = self.gru(x, self.h0)
        x = self.map(x)
        return self.f(x)
    
    def init_hidden_state(self):
        self.h0 = torch.zeros(1,1,self.hidden_size)
        return self.h0


class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Discriminator, self).__init__()
        self.hidden_size = hidden_size
        self.h0 = self.init_hidden_state()
        self.gru = nn.GRU(input_size, hidden_size)
        
        #self.map = nn.Linear(input_size, + hidden_size, 1)
        
        #this line works
        self.map = nn.Linear(input_size, 1) # + hidden_size, 1)
        self.f = f

    def forward(self, x):
        x, self.h0 = self.gru(x, self.h0)
        #print(x.shape)
        #print(self.h0.shape)
        #x = self.map(torch.cat((x, self.h0), 1), 1)
        
        #this line works
        x = self.map(x) #torch.cat((x, self.h0),1)) #should we run hidden state through last layer?
        return self.f(x)
    
    def init_hidden_state(self):
        self.h0 = torch.zeros(1,1,self.hidden_size)
        return self.h0
 

#class Discriminator(nn.Module):
#    def __init__(self, input_size, hidden_size, output_size, f):
#        super(Discriminator, self).__init__()
#        self.map1 = nn.Linear(input_size, hidden_size)
#        self.map2 = nn.Linear(hidden_size, hidden_size)
#        self.map3 = nn.Linear(hidden_size, output_size)
#        self.f = f

#    def forward(self, x):
#        x = self.f(self.map1(x))
#        x = self.f(self.map2(x))
#        return self.f(self.map3(x))

In [83]:
#Define Generator and Discriminator
G = Generator(X_seq.shape[2], X_seq.shape[2], X_seq.shape[2], torch.sigmoid)
D = Discriminator(X_seq.shape[2], X_seq.shape[2], 1, torch.sigmoid)

#testing how resetting the hidden state has an effect on the output
G.zero_grad()
G.init_hidden_state()
first = G(torch.tensor(X_seq).clone())
print(first)

G.zero_grad()
G.init_hidden_state()
second = G(torch.tensor(X_seq).clone())
print(second)

#these outputs should be the same

tensor([[[0.5279, 0.4916, 0.4952, 0.4857, 0.5182, 0.4988, 0.4546, 0.4814,
          0.5026, 0.5092, 0.4649, 0.4880, 0.5076, 0.4773, 0.5022, 0.5310,
          0.5509, 0.5128, 0.5184, 0.4858, 0.5002, 0.5383, 0.5059, 0.5118,
          0.4960, 0.4749, 0.5133, 0.5578, 0.5086, 0.5191, 0.4956, 0.4786,
          0.4935, 0.4894, 0.5120, 0.5162, 0.5140, 0.4965, 0.5277, 0.4513,
          0.4963, 0.4820, 0.4775, 0.4999, 0.4905, 0.4950, 0.5483, 0.5314,
          0.4891, 0.5121, 0.5211, 0.4896, 0.4923, 0.4799, 0.5175, 0.5375,
          0.5194, 0.4898, 0.4648, 0.5365, 0.4863, 0.4751, 0.4902, 0.5126,
          0.5092, 0.5140, 0.4733, 0.5103, 0.4975, 0.4960, 0.5042, 0.4737,
          0.4989, 0.5115, 0.5158, 0.4542, 0.5007, 0.4752, 0.5031, 0.5489,
          0.4799, 0.4995, 0.5017, 0.4935, 0.5140, 0.4637, 0.5234, 0.5011,
          0.4858, 0.4991, 0.4920, 0.4854, 0.5417, 0.5373, 0.4506, 0.4942]],

        [[0.5306, 0.4919, 0.4876, 0.4749, 0.5294, 0.4932, 0.4400, 0.4743,
          0.4999, 0.5062, 0.4591, 0

/home/jaywalker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/jaywalker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


In [78]:
import torch.optim as optim 

def get_generator_input_sampler():
    return lambda m, n: torch.rand(m, n)  # Uniform-dist data into generator, _NOT_ Gaussian

def extract(v):
    return v.data.storage().tolist()

#well OK let's train the GAN on a single sequence and see what happens
def train(G, D, real_data):
    num_epochs = 10000
    print_interval = 1000
    print_stats = True
    
    discriminator_learning_rate = 1e-3
    generator_learning_rate = 1e-3
    sgd_momentum = 0.9
    
    discriminator_training_steps = 1
    generator_training_steps = 1
    
    discriminator_fake_error, discriminator_real_error = 0.0, 0.0
    generator_error = 0
    
    criterion = nn.BCELoss() #right now the output is binary so this makes sense
    discriminator_optimizer = optim.SGD(D.parameters(), lr=discriminator_learning_rate, momentum=sgd_momentum)
    generator_optimizer = optim.SGD(G.parameters(), lr=discriminator_learning_rate, momentum=sgd_momentum)
    
    sequence_length = real_data.shape[0]
    for epoch in range(num_epochs):
        for discriminator_step in range(discriminator_training_steps):
            D.init_hidden_state()
            D.zero_grad()
            
            #Train D on the real samples
            #print(real_data.shape)
            discriminator_decision_r = D(real_data)
            #print(discriminator_decision_r.shape)
            discriminator_real_error = criterion(discriminator_decision_r, torch.ones(sequence_length))
            discriminator_real_error.backward()
            
            #Train D on the fake samples
            D.init_hidden_state()
            #D.zero_grad()
            
            #create a sample of length-1 real samples and generate the last
            generator_input = real_data[:sequence_length - 2, :, :]
            #print("Generator input shape: ", generator_input.shape)
            fake_data = torch.zeros(sequence_length - 1, 1, real_data.shape[2])
            fake_data[:sequence_length - 2, :, :] = generator_input
            
            fake_data_element = G(generator_input).detach()
            #print("Fake data element shape: ", fake_data_element.shape)
            #print("Fake data shape: ", fake_data.shape)
            fake_data[sequence_length - 2, :, :] = fake_data_element[2, :, :] #generate the last packet in the sequence
            discriminator_decision_f = D(fake_data)
            discriminator_fake_error = criterion(discriminator_decision_f, torch.zeros(sequence_length - 1))
            discriminator_fake_error.backward()
            discriminator_optimizer.step()
            
            dre = extract(discriminator_real_error)[0]
            dfe = extract(discriminator_fake_error)[0]
    
        for generator_step in range(generator_training_steps):
            pass #print("generator step stub")
            
            
        if print_stats:
            if epoch % print_interval == 0 or epoch == num_epochs-1:
                print("D Real Error: ", dre)
                print("D Fake Error: ", dfe)
            
            
train(G,D,X_seq)

D Real Error:  0.4865334630012512
D Fake Error:  0.4417358934879303
D Real Error:  0.4865504205226898
D Fake Error:  0.44163066148757935
D Real Error:  0.4865618646144867
D Fake Error:  0.44154608249664307
D Real Error:  0.4865693151950836
D Fake Error:  0.44147738814353943
D Real Error:  0.4865745007991791
D Fake Error:  0.44142040610313416
D Real Error:  0.48657768964767456
D Fake Error:  0.4413728713989258
D Real Error:  0.4865804612636566
D Fake Error:  0.4413316249847412
D Real Error:  0.48658180236816406
D Fake Error:  0.44129666686058044
D Real Error:  0.48658281564712524
D Fake Error:  0.44126591086387634
D Real Error:  0.486583411693573
D Fake Error:  0.4412389397621155
D Real Error:  0.4865836501121521
D Fake Error:  0.44121524691581726


In [ ]:
print(G(X_seq).detach())